In [1]:
import json
import csv
from collections import OrderedDict
import urllib.request
import time
import psycopg2
import os
import re
import numpy as np

## Getting and Processing Data

In [2]:
#Prevent webpage blocking on urlopen
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
url = 'https://remoteok.io/remote-jobs.json'
headers={'User-Agent':user_agent,} 
request=urllib.request.Request(url,None,headers) #The assembled request

In [3]:
#Request data in terms of json from remote.ok webpage
res = urllib.request.urlopen(request) #res is an object of http request

#Decode object of client service into json 
raw_data = res.read()
encoding =res.info().get_content_charset('utf8')

In [4]:
#Load decoded data in order as json format
data = json.loads(raw_data.decode(encoding), object_pairs_hook=OrderedDict) 

In [5]:
#strips all non-ascii characters
#this is for all remaining non-significant ones
def make_ascii(text):
    return ''.join([i if ord(i) < 128 else '' for i in text])

#strips out all html markup and insert ascii replacements
def cleanhtml(raw_html):
    #replace for \n
    raw_html = raw_html.replace("<br>","\n")
    raw_html = raw_html.replace("</li>","\n")
    raw_html = re.sub("</h*>","\n", raw_html)
    raw_html = re.sub("</h3>","\n", raw_html)
    #html conversions
    raw_html = raw_html.replace("&nbsp;"," ")
    raw_html = raw_html.replace("	"," ")
    raw_html = raw_html.replace("&#x27;","'")
    raw_html = raw_html.replace("â","'")    
    raw_html = raw_html.replace("â","-")
    raw_html = raw_html.replace("â","-")
    raw_html = raw_html.replace("â¢","-")
    raw_html = raw_html.replace("â","-")
    raw_html = raw_html.replace("&lt;","<")
    raw_html = raw_html.replace("&gt;",">")
    raw_html = raw_html.replace("â¦","...")
    raw_html = raw_html.replace("â¦","...")
    raw_html = raw_html.replace("&amp","and")
    raw_html = raw_html.replace("and;","and")
    Raw_html = raw_html.replace("&","and")
    raw_html = raw_html.replace("Ã©","e")
    raw_html = raw_html.replace("â¬","€")
    raw_html = raw_html.replace("â","\"") 
    raw_html = raw_html.replace("â","\"")       
    raw_html = raw_html.replace("Â","")
    raw_html = raw_html.replace("__","")
    raw_html = raw_html.replace("_","")
    raw_html = raw_html.replace("ï¿½","")
    raw_html = re.sub("##+","",raw_html)
    #remove noise markups
    cleantext = raw_html
    cleantext = re.sub('<.*?>','',cleantext)
    cleantext = re.sub(' +',' ',cleantext)
    cleantext = re.sub('(\n)(\n)(\n)+','\n\n',cleantext)
    return make_ascii(cleantext)

#iterate through listing descriptions and clean each one
for job_number, job in enumerate(data):   
    job["description"] = cleanhtml(str(job["description"]))
    job["label"] = 0

In [6]:
#To see sample data
for job_number, job in enumerate(data):
    for item in job:
        print(item)
        print(job[item])
    break

slug
60907-remote-user-researcher-authoring-tools-ink-switch
id
60907
epoch
1500564066
date
2017-07-20T08:21:06-07:00
company
Ink Switch
position
User Researcher Authoring Tools
tags
[]
logo

description
Ink & Switch is hiring a User Researcher, Authoring Tools anywhere
url
https://remoteok.io/jobs/60907-remote-user-researcher-authoring-tools-ink-switch
label
0


## CSV Generation


In [7]:
#Create file name with the current date for daily based update
timestr = time.strftime("%Y%m%d") 
file_name='D'+str(timestr)+"_raw"
print(file_name)

D20170720_raw


In [8]:
#Convert json file to csv file
keys = data[0].keys()
with open("./CSVs/"+file_name + '.csv', 'w', encoding='utf-8', newline = '') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    count = 1
    for d in data:
        dict_writer.writerow(d)
        count += 1

## PostgreSQL

In [9]:
#Connect to postgreSQL remote
conn = psycopg2.connect(host="192.168.1.67", database="Remote_ok_db", user="postgres", password="password", port=5432)
cur = conn.cursor()

In [11]:
#Connect to postgreSQL local
conn = psycopg2.connect(host="localhost", database="Remote_ok_db", user="postgres", password="password")
cur = conn.cursor()

In [10]:
#Connect to postgreSQL remote
conn = psycopg2.connect(host="192.168.1.67", database="Remote_ok_db", user="postgres", password="password", port=5432)
cur = conn.cursor()

In [11]:
#Create tables in the PostgreSQL database
sql_exe=""" CREATE TABLE """+" "+file_name+"""
( slug varchar,
  id character(5) NOT NULL,
  epoch character(10),
  postdate timestamp with time zone,
  company varchar,
  position varchar,
  tags varchar,
  logo varchar(2083),
  description varchar,
  url varchar(2083),
  label varchar
);"""
#execute SQL query
cur.execute(sql_exe)
print('Created table')

Created table


In [12]:
base_dir = os.path.join(os.getcwd(),"CSVs")
path_name= file_name+'.csv'
path=os.path.join(base_dir,path_name)

sql1 = 'COPY' +" "+file_name+' '+'FROM stdin'
sql2="""
WITH CSV HEADER 
DELIMITER as ',' """ 
copy_sql= sql1 + sql2

In [13]:
#Import data into table
with open(path, 'r', encoding='utf-8') as f:
    cur.copy_expert(sql=copy_sql, file=f)
    print('Successfully imported data')
    conn.commit()    

Successfully imported data
